## LOAD EXAMPLE DATA FROM RETAIL TABLE

In [1]:
import sys
import os

# Get the parent directory of the current notebook (project_root)
notebook_dir = os.path.dirname(os.path.abspath('__file__'))
project_root = os.path.abspath(os.path.join(notebook_dir, '..'))

# Add the project root directory to the Python path
sys.path.append(project_root)



In [2]:
import numpy as np 
import pandas as pd 
import chardet



# Load the CSV data into a DataFrame
csv_file = 'Online Retail Data Set.csv'  # Replace with the actual path to your CSV file

with open(csv_file, 'rb') as f:
    result = chardet.detect(f.read())
    
data = pd.read_csv(csv_file, encoding=result['encoding'])


# Display basic information about the DataFrame
print("DataFrame Info:")
print(data.info())

# Display the first few rows of the DataFrame
print("\nFirst few rows of the DataFrame:")
print(data.head())


DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB
None

First few rows of the DataFrame:
  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BO

In [17]:
import pandas as pd

# Create or load your original DataFrame
# df = ...

# Get the unique countries from the "country" column
unique_countries = data["Country"].unique()

# Randomly select one user from each unique country
selected_rows = []
for country in unique_countries:
    country_df = data[data["Country"] == country]
    if not country_df.empty:
        selected_row = country_df.sample(n=1, random_state=42)
        selected_rows.append(selected_row)

# Concatenate the selected rows to create the mini version of the DataFrame
mini_df = pd.concat(selected_rows)


In [4]:
from main.models import ModelClass

txt = ModelClass()